In [25]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE, mean_absolute_error

In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost
import optuna

In [26]:
# Загрузим данные. Я их немножно подправил ещё до загрузки.
df = pd.read_excel("data/data_00.xlsx")

# Уберём ненужные столбцы и пустые значения
df = df.drop(columns=["Номер образца", "Адсорбент"]).dropna()

# Ещё один нюанс
df = df[df["Sme, м2/г"] != "-"]

# Сменим тип данных у таблицы
df["Sme, м2/г"] = df["Sme, м2/г"].astype(np.float64, copy=False)
df["m (соли), г"] = df["m (соли), г"].astype(np.float64, copy=False)
df["Vпр. (р-ля), мл"] = df["Vпр. (р-ля), мл"].astype(np.float64, copy=False)

# У нас есть 3 категориальных признака, которых мы закодируем числами
list_of_cats = ["Металл", "Лиганд", "Растворитель"]
cat2id = {cat:{v:i for i, v in enumerate(df[cat].drop_duplicates().values)} for cat in list_of_cats}
id2cat = {cat:{i:v for i, v in enumerate(df[cat].drop_duplicates().values)} for cat in list_of_cats}
for cat in list_of_cats:
    df[cat] = df[cat].apply(lambda x: cat2id[cat][x])

In [27]:
y_cols = ['W0, см3/г', 'SБЭТ, м2/г']
X_cols = ['Tрег, ᵒС\n', 'Металл', 'Лиганд', 'Растворитель', 'm (соли), г', 'm(кис-ты), г', 'Vсин. (р-ля), мл', 'Т.син., °С', 'Vпр. (р-ля), мл', 'Т суш., °С']

In [28]:
X = df[X_cols]

In [29]:
y1 = df[y_cols[0]]
y2 = df[y_cols[1]]

#### W0

In [11]:
y = y1

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model
model = xgboost.XGBRegressor()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }
    model = xgboost.XGBRegressor(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred)

# Create the study
study = optuna.create_study(direction='minimize', study_name='regression')
study.optimize(objective, n_trials=100)

# Print the best parameters
print('Best parameters', study.best_params)

In [55]:
model = xgboost.XGBRegressor(**study.best_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

MSE:  0.011339451423005737
RMSE:  0.1064868603303043


In [57]:
import pickle

In [58]:
filename = "xgbregressor_01.pkl"
pickle.dump(model, open(filename, "wb"))

#### SБЭТ

In [59]:
y = y2

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model
model = xgboost.XGBRegressor()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }
    model = xgboost.XGBRegressor(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred)

# Create the study
study = optuna.create_study(direction='minimize', study_name='regression')
study.optimize(objective, n_trials=100)

# Print the best parameters
print('Best parameters', study.best_params)

[I 2023-01-14 08:53:03,198] A new study created in memory with name: regression


MSE:  123820.10928194581
RMSE:  351.880816871204


[I 2023-01-14 08:53:07,087] Trial 0 finished with value: 264663.5778000807 and parameters: {'max_depth': 8, 'learning_rate': 0.8633323750736667, 'n_estimators': 675, 'min_child_weight': 4, 'gamma': 0.7100649961692896, 'subsample': 0.6478455017335141, 'colsample_bytree': 0.2288646938868661, 'reg_alpha': 0.32400652928395607, 'reg_lambda': 0.6226288679183601, 'random_state': 658}. Best is trial 0 with value: 264663.5778000807.
[I 2023-01-14 08:53:12,366] Trial 1 finished with value: 148779.84331092186 and parameters: {'max_depth': 7, 'learning_rate': 0.8568205547713101, 'n_estimators': 631, 'min_child_weight': 5, 'gamma': 0.43002468829799145, 'subsample': 0.9800983387779272, 'colsample_bytree': 0.9024161495420594, 'reg_alpha': 0.11195855594890738, 'reg_lambda': 0.05336413749338706, 'random_state': 495}. Best is trial 1 with value: 148779.84331092186.
[I 2023-01-14 08:53:18,806] Trial 2 finished with value: 201579.9423180353 and parameters: {'max_depth': 10, 'learning_rate': 0.744689297069

Best parameters {'max_depth': 6, 'learning_rate': 0.06165509560718764, 'n_estimators': 160, 'min_child_weight': 4, 'gamma': 0.37805003263571363, 'subsample': 0.7916041117470214, 'colsample_bytree': 0.6280260719762699, 'reg_alpha': 0.7956612584272865, 'reg_lambda': 0.8530080796733946, 'random_state': 995}


In [61]:
model = xgboost.XGBRegressor(**study.best_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

MSE:  78584.07873303235
RMSE:  280.3285193001817


In [62]:
filename = "xgbregressor_02.pkl"
pickle.dump(model, open(filename, "wb"))

In [64]:
filename = "scaler.pkl"
pickle.dump(scaler, open(filename, "wb"))